In [1]:
import numpy as np
import pandas as pd
import os
# cuda error 표시 안될 때 
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import torch
import sklearn
import torchvision
from tqdm import tqdm

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# set seed
torch.manual_seed(1)
if device == "cuda":
    torch.cuda.manual_seed_all(1)

In [4]:
# csv load
import os
data_path = "./2021-ai-w11-p1/"
labelname_csv_path = os.path.join(data_path,"Label2Names.csv")
labelname = pd.read_csv(labelname_csv_path,header=None)
print(labelname)
train_dir_path = os.path.join(data_path,"train_csv_v2")
test_dir_path = os.path.join(data_path,"test_csv_v2")
submission = pd.read_csv(os.path.join(data_path,"submission.csv"))
print(submission)

       0              1
0      1          Faces
1      2     Faces_easy
2      3       Leopards
3      4     Motorbikes
4      5      accordion
..   ...            ...
96    97     wheelchair
97    98       wild_cat
98    99  windsor_chair
99   100         wrench
100  101       yin_yang

[101 rows x 2 columns]
                 Id  Category
0     img_00000.csv        42
1     img_00001.csv        42
2     img_00002.csv        42
3     img_00003.csv        42
4     img_00004.csv        42
...             ...       ...
1707  img_01707.csv        42
1708  img_01708.csv        42
1709  img_01709.csv        42
1710  img_01710.csv        42
1711  img_01711.csv        42

[1712 rows x 2 columns]


In [5]:
def load_train_data(dir_path,labelname):
    train_x = []
    train_y = []
    for dir_name in tqdm(os.listdir(dir_path)):
        # dirname to label
        if dir_name == "BACKGROUND_Google":
            continue
        else:
            label = labelname.index[labelname[1] == dir_name].tolist()[0] + 1
            for csv_name in os.listdir(os.path.join(dir_path,dir_name)):
                csv_path = os.path.join(dir_path,dir_name,csv_name)
                # reshape 1d to 2d
                train_x.append(np.array(pd.read_csv(csv_path)).reshape((256,256,3)).transpose((2,0,1)))
                train_y.append(label)
    return train_x,train_y

In [6]:
# load train_x,train_y
train_x, train_y = load_train_data(train_dir_path,labelname)
print(len(train_x),len(train_y))

100%|██████████| 102/102 [00:39<00:00,  2.57it/s]

3030 3030


In [7]:
# # data scaling
# from sklearn.preprocessing import StandardScaler
# train_x = np.array(train_x)
# train_y = np.array(train_y)
# test_x = np.array(test_x)

# scaler = StandardScaler()
# train_x = scaler.fit_transform(train_x)
# test_x = scaler.transform(test_x)

In [8]:
# data reshape 1D -> 2D(3channel)
train_x = np.array(train_x)
train_y = np.array(train_y)
# train_x = train_x.reshape((len(train_x),256,256,3))
# train_x = train_x.transpose(0,3,1,2)
# test_x = test_x.reshape((len(test_x),256,256,3))
# test_x = test_x.transpose(0,3,1,2)
print(train_x.shape)
print(train_y.shape)

(3030, 3, 256, 256)
(3030,)


In [9]:
# data to tensor , dataset
from torch.utils.data import DataLoader,TensorDataset
train_dataset = TensorDataset(torch.FloatTensor(train_x),torch.LongTensor(train_y))

In [10]:
# memory clear
del train_x
del train_y

In [11]:
# model load
model = torchvision.models.vgg19(pretrained=True)
for param in model.parameters():
    param.requires_grad = False    
model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=101, bias=True)
# init xavier
torch.nn.init.xavier_normal_(model.classifier[0].weight)
torch.nn.init.xavier_normal_(model.classifier[3].weight)
torch.nn.init.xavier_normal_(model.classifier[6].weight)
model = model.to(device)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [12]:
# setting param, optim, cost function,dataloader
from torch.utils.data import DataLoader

lr = 1e-3
batch_size = 16
optim = torch.optim.Adam(model.parameters(), lr=lr)
train_data_loader = DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
epochs = 100
loss = torch.nn.CrossEntropyLoss()

In [13]:
def train(model, data_loader):
    model.train()
    sum_cost = 0.0
    sum_correct = 0
    for i,data in enumerate(data_loader):
        data, target = data[0].to(device),data[1].to(device)
        optim.zero_grad()
        output = model(data)
        cost = loss(output,target - 1)
        predict = torch.argmax(output,dim=1)
        correct = (predict == target - 1).sum().item()
        sum_cost += cost.item()
        sum_correct += correct
        cost.backward()
        optim.step()
    return sum_cost/len(data_loader.dataset),sum_correct/len(data_loader.dataset)

In [14]:
#train 
import time
cur_time = time.time()
for epoch in range(epochs):
    cost,acc = train(model,train_data_loader)
    print(epoch,cost,acc * 100)
print("endtime =",time.time() - cur_time)

c:\python39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


0 3.6775129802943063 4.818481848184819
1 2.4356997002076 18.05280528052805
2 1.8269342723065871 30.363036303630363
3 1.6241819150377028 36.36963696369637


KeyboardInterrupt: 

In [15]:
def validate(model, data_loader):
    model.eval()
    sum_correct = 0
    for i,data in enumerate(data_loader):
        output = model(data[0].to(device))
        target = data[1].to(device)
        predict = torch.argmax(output,dim=1)
        correct = (predict == target - 1).sum().item()
        sum_correct += correct
    return sum_correct/len(data_loader.dataset)

In [16]:
with torch.no_grad():
    print(validate(model,train_data_loader) * 100)

77.35973597359735


In [17]:
def load_test_data(dir_path):
    test_x = []
    sort_csv_names = sorted(os.listdir(dir_path))
    for csv_name in tqdm(sort_csv_names):
        csv_path = os.path.join(dir_path,csv_name)
        test_x.append(np.array(pd.read_csv(csv_path)).reshape((256,256,3)).transpose((2,0,1)))
    return test_x

In [18]:
# load test_x
test_x = load_test_data(test_dir_path)
test_x = np.array(test_x)
test_dataset = TensorDataset(torch.FloatTensor(test_x))
test_data_loader = DataLoader(dataset=test_dataset, batch_size = batch_size, shuffle=False)

100%|██████████| 1712/1712 [00:22<00:00, 75.86it/s]


In [19]:
def predict(model, data_loader):
    model.eval()
    predict_array = []
    for data in (data_loader):
        output = model(data[0].to(device))
        predict = torch.argmax(output,dim=1) + 1
        predict_array += np.array(predict.cpu().detach()).tolist()
    return predict_array

In [20]:
with torch.no_grad():
    predict = predict(model,test_data_loader)
    submission['Category'] = predict
    print(submission)

                 Id  Category
0     img_00000.csv        52
1     img_00001.csv        11
2     img_00002.csv        52
3     img_00003.csv        42
4     img_00004.csv        42
...             ...       ...
1707  img_01707.csv        97
1708  img_01708.csv        29
1709  img_01709.csv        55
1710  img_01710.csv        88
1711  img_01711.csv        98

[1712 rows x 2 columns]


In [ ]:
submission.to_csv("submission.csv",index=False)